In [2]:
%pip install hdbscan


   ---------------------------------------- 0.0/732.2 kB ? eta -:--:--
   --------------------------------------- 732.2/732.2 kB 10.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Daten und Vektoren
import pandas as pd
import numpy as np

# NLP & Vektoren laden
import spacy
from spacy.tokens import DocBin

# Clustering
import hdbscan
from sklearn.decomposition import PCA

# Visualisierung
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
# Zeichenanzahl pro Spalte erhöhen
pd.set_option("display.max_colwidth", None)


In [3]:
# spaCy-Modell laden
nlp = spacy.load("de_core_news_lg")

# Docs laden (wie vorher gespeichert)
doc_bin = DocBin().from_disk("docs_de.spacy")
docs = list(doc_bin.get_docs(nlp.vocab))

# DataFrame laden
df = pd.read_excel("../../data/df_de_final_duplikate_markiert.xlsx")


In [4]:
# Vektoren extrahieren
vectors = np.array([doc.vector for doc in docs])


from sklearn.metrics.pairwise import cosine_distances

# Cosine-Distanzmatrix erzeugen und in float64 casten
distance_matrix = cosine_distances(vectors).astype(np.float64)


In [5]:
# HDBSCAN mit Cosine-Distanz
clusterer = hdbscan.HDBSCAN(min_cluster_size=2, metric='precomputed')
labels = clusterer.fit_predict(distance_matrix)


# Labels ins DataFrame schreiben
df["HDBSCAN_Cluster"] = labels

In [6]:
print(df["HDBSCAN_Cluster"].value_counts().sort_index())


HDBSCAN_Cluster
-1       1538
 0          3
 1          2
 2          8
 3         18
         ... 
 1003      10
 1004       3
 1005       3
 1006       4
 1007       4
Name: count, Length: 1009, dtype: int64


In [7]:
df[df["HDBSCAN_Cluster"] == 300][["Frage_Text"]].head(10)


,Frage_Text
3473,"Sollen Kinder, die nicht Deutsch als Muttersprache haben, zuerst in separaten Klassen unterrichtet werden (vor Integration in Regelklasse)?"
3517,"Sollen Kinder, die nicht Deutsch als Muttersprache haben, zuerst in separaten Klassen unterrichtet werden (vor Integration in Regelklasse)?"


In [8]:
df[df["HDBSCAN_Cluster"] == 27][["Frage_Typ"]].head()


,Frage_Typ
76,options4
4987,Standard-4
6537,Standard-4


In [9]:
df[df["HDBSCAN_Cluster"] == -1][["Frage_Text"]].head(10)


,Frage_Text
6,"Wie beurteilen Sie diese Aussage: ""Vermögende sollen sich stärker an der Finanzierung des Staates beteiligen."""
268,"Soll sich die Stadt Luzern beim Kanton für eine Verschärfung des Sozialhilfegesetzes einsetzen (z.B. Begrenzung der Zulagen, tieferes Existenzminimum, strengere Sanktionen)?"
271,Befürworten Sie die Einführung eines bezahlten Vaterschaftsurlaubs von zwei Wochen?
279,Soll sich die Stadt Luzern stärker für Asylsuchende engagieren (bspw. im Rahmen der sog. 'Sanctuary City')?
282,Befürworten Sie eine Reduktion der städtischen Kultursubventionen?
283,Befürworten Sie eine Kürzung des Beitrags an das Luzerner Fest 2021?
289,Befürworten Sie einen Einstellungsstopp für die Stadt Luzern (Einfrieren des Stellenbestands)?
292,"Soll die Stadt Luzern das lokale Gewerbe stärker unterstützen (z.B. schnellere Bewilligungsverfahren, keine Parkplatzreduktion)?"
295,Soll die Stadt Luzern den Reisecar-Tourismus stärker regulieren (z.B. Einführung einer Gebühr für Halteplätze der Reisecars)?
299,Sollte der motorisierte Privatverkehr in der städtischen Verkehrspolitik eine höhere Priorität erhalten?


In [10]:
from collections import Counter

# Alle eindeutigen Cluster (ohne -1 / Noise)
cluster_ids = df["HDBSCAN_Cluster"].unique()
cluster_ids = [cid for cid in cluster_ids if cid != -1]

# Dictionary zum Speichern der Top-Wörter pro Cluster
cluster_topwörter = {}

# Für jedes Cluster: Wörter zählen & speichern
for cluster_id in cluster_ids:
    fragen = df[df["HDBSCAN_Cluster"] == cluster_id]["Frage_Text"]
    wörter = []
    
    for frage in fragen:
        doc = nlp(frage)
        for token in doc:
            if not token.is_stop and not token.is_punct and token.is_alpha:
                wörter.append(token.lemma_.lower())
    
    counter = Counter(wörter)
    häufigste = counter.most_common(10)
    # Formatieren: wort: anzahl
    cluster_topwörter[cluster_id] = ", ".join([f"{w}: {c}" for w, c in häufigste])

# Neue Spalte mit den Top-Wörtern
df["Cluster_Top_Wörter"] = df["HDBSCAN_Cluster"].apply(
    lambda cid: cluster_topwörter.get(cid, "Noise / kein Cluster")
)


In [11]:
df.iloc[1145]

ID_Wahl                                                                                                                            67.0
Datum                                                                                                               2021-02-07 00:00:00
Frage_ID                                                                                                                         2816.0
Frage_Text            Soll die 30-Jahres-Frist für eine erleichterte Einbürgerung von alteingesessenen Ausländer/-innen gekürzt werden?
Frage_Typ                                                                                                                      options4
Bereich_ID                                                                                                                       4282.0
Bereich                                                                                                         Migration & Integration
ID_gesamt                                       

In [12]:
# DataFrame als Excel-Datei exportieren
df.to_excel("../../data/df_de_final_duplikate_markiert_HDBSCAN.xlsx", index=False, engine="openpyxl")


In [13]:
df[(df["HDBSCAN_Cluster"] == -1) & (df["Duplikat"] == True)][["Frage_Text"]]


,Frage_Text
6,"Wie beurteilen Sie diese Aussage: ""Vermögende sollen sich stärker an der Finanzierung des Staates beteiligen."""
462,"Soll das Angebot im Bereich der Kinderdrittbetreuung in der Stadt Biel ausgebaut werden (Erhöhung der Krippenplätze, Verlängerung der Öffnungszeiten von Tagesschulen, usw.)?"
526,Soll der Kanton die Lohngleichheit von Frauen und Männern stärker kontrollieren?
533,"Soll der Kanton Schaffhausen das Service-Public-Angebot (z.B. ÖV-Verbindungen, Poststellen) stärker fördern?"
534,"Soll der Kanton Kadermitarbeiter/-innen oder bestimmten Fachpersonen weiterhin Zulagen zahlen, um die Attraktivität des Kantons als Arbeitgeber zu steigern?"
...,...
7436,Würden Sie der Schaffung einer kantonalen Verwaltungsstelle zur Verbesserung der Gleichstellung von Menschen mit Behinderung befürworten?
7439,"Soll der Kanton Zürich die Schaffung von familienergänzenden Betreuungsstrukturen (Tagesstätten, Tagesschulen, Mittagstische) verstärkt finanziell unterstützen?"
7443,"Würden Sie eine Verschärfung des Sozialhilfegesetzes im Kanton Zürich (z.B. Begrenzung der Zulagen, tieferer Ansatz des Existenzminimums, höherer Ermessenspielraum bei der Vergabe der Sozialhilfe) befürworten?"
7449,"Sollen Landwirte, welche nicht umweltfreundlich produzieren (bspw. präventiver Einsatz von Pestiziden oder Antibiotika), keine Direktzahlungen mehr erhalten?"


In [14]:
df[df["Duplikat_Gruppe"] == "d717627dc6d0d8b3c09432f355181f17"]


,ID_Wahl,Datum,Frage_ID,Frage_Text,Frage_Typ,Bereich_ID,Bereich,ID_gesamt,Sprache,Duplikat,Frage_Hash,Duplikat_Gruppe,HDBSCAN_Cluster,Cluster_Top_Wörter
6,2.0,2019-10-20 00:00:00,31.0,"Wie beurteilen Sie diese Aussage: ""Vermögende sollen sich stärker an der Finanzierung des Staates beteiligen.""",options7,4734.0,Werthaltungen,Q00006,de,True,d717627dc6d0d8b3c09432f355181f17,d717627dc6d0d8b3c09432f355181f17,-1,Noise / kein Cluster
361,49.0,2020-09-27 00:00:00,1388.0,"Wie beurteilen Sie diese Aussage: ""Vermögende sollen sich stärker an der Finanzierung des Staates beteiligen.""",options7,4781.0,Werthaltungen,Q00361,de,True,d717627dc6d0d8b3c09432f355181f17,d717627dc6d0d8b3c09432f355181f17,250,"beurteilen: 21, aussage: 21, vermögende: 21, stark: 21, finanzierung: 21, staat: 21, beteiligen: 21"
410,18.0,2020-09-27 00:00:00,1837.0,"Wie beurteilen Sie diese Aussage: ""Vermögende sollen sich stärker an der Finanzierung des Staates beteiligen.""",options7,4768.0,Werthaltungen,Q00410,de,True,d717627dc6d0d8b3c09432f355181f17,d717627dc6d0d8b3c09432f355181f17,250,"beurteilen: 21, aussage: 21, vermögende: 21, stark: 21, finanzierung: 21, staat: 21, beteiligen: 21"
456,51.0,2020-09-27 00:00:00,1633.0,"Wie beurteilen Sie diese Aussage: ""Vermögende sollen sich stärker an der Finanzierung des Staates beteiligen.""",options7,4779.0,Werthaltungen,Q00456,de,True,d717627dc6d0d8b3c09432f355181f17,d717627dc6d0d8b3c09432f355181f17,250,"beurteilen: 21, aussage: 21, vermögende: 21, stark: 21, finanzierung: 21, staat: 21, beteiligen: 21"
503,20.0,2020-09-27 00:00:00,1141.0,"Wie beurteilen Sie diese Aussage: ""Vermögende sollen sich stärker an der Finanzierung des Staates beteiligen.""",options7,4756.0,Werthaltungen,Q00503,de,True,d717627dc6d0d8b3c09432f355181f17,d717627dc6d0d8b3c09432f355181f17,251,"beurteilen: 3, aussage: 3, vermögende: 3, stark: 3, finanzierung: 3, staat: 3, beteiligen: 3"
551,22.0,2020-09-27 00:00:00,1902.0,"Wie beurteilen Sie diese Aussage: ""Vermögende sollen sich stärker an der Finanzierung des Staates beteiligen.""",options7,4761.0,Werthaltungen,Q00551,de,True,d717627dc6d0d8b3c09432f355181f17,d717627dc6d0d8b3c09432f355181f17,251,"beurteilen: 3, aussage: 3, vermögende: 3, stark: 3, finanzierung: 3, staat: 3, beteiligen: 3"
609,24.0,2020-10-18 00:00:00,2155.0,"Wie beurteilen Sie diese Aussage: ""Vermögende sollen sich stärker an der Finanzierung des Staates beteiligen.""",options7,4757.0,Werthaltungen,Q00609,de,True,d717627dc6d0d8b3c09432f355181f17,d717627dc6d0d8b3c09432f355181f17,251,"beurteilen: 3, aussage: 3, vermögende: 3, stark: 3, finanzierung: 3, staat: 3, beteiligen: 3"
670,45.0,2020-10-25 00:00:00,2335.0,"Wie beurteilen Sie diese Aussage: ""Vermögende sollen sich stärker an der Finanzierung des Staates beteiligen.""",options7,4784.0,Werthaltungen,Q00670,de,True,d717627dc6d0d8b3c09432f355181f17,d717627dc6d0d8b3c09432f355181f17,247,"beurteilen: 5, aussage: 5, vermögende: 5, stark: 5, finanzierung: 5, staat: 5, beteiligen: 5"
824,32.0,2020-11-29 00:00:00,2796.0,"Wie beurteilen Sie diese Aussage: ""Vermögende sollen sich stärker an der Finanzierung des Staates beteiligen.""",options7,4760.0,Werthaltungen,Q00824,de,True,d717627dc6d0d8b3c09432f355181f17,d717627dc6d0d8b3c09432f355181f17,247,"beurteilen: 5, aussage: 5, vermögende: 5, stark: 5, finanzierung: 5, staat: 5, beteiligen: 5"
872,53.0,2021-03-07 00:00:00,2989.0,"Wie beurteilen Sie diese Aussage: ""Vermögende sollen sich stärker an der Finanzierung des Staates beteiligen.""",options7,4787.0,Werthaltungen,Q00872,de,True,d717627dc6d0d8b3c09432f355181f17,d717627dc6d0d8b3c09432f355181f17,247,"beurteilen: 5, aussage: 5, vermögende: 5, stark: 5, finanzierung: 5, staat: 5, beteiligen: 5"
